#  Build ETL Processes
Developpement ETL processes for each table.

In [1]:
# Importation the library for processing
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [5]:
# Connecton to the databaset
try:
    conn = psycopg2.connect("host = 127.0.0.1 dbname = sparkifydb user = postgres password = 1234")
    cur = conn.cursor()
except psycopg2.Error as e:
    print('Error')
    print(e)

In [8]:
def get_files(filpath):
    all_files = []
    for root, dirs, files in os.walk(filpath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files

#  Process song_data
In this firt part, we will perform ETL on the first dataset song_data, to create the songs and artists dimenional tables.


In [9]:
song_files = get_files('./data/song_data')


In [12]:
filepath = song_files[0]

In [13]:
df = pd.read_json(filepath, lines = True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


#  Songs Table
Extract Data for Songs Table
 - Select columns for song ID, title, artist ID, Year..
 - Index to select the first(only) record in the dataframe
 

In [25]:
song_data = list(df[['song_id','title', 'artist_id', 'year', 'duration']].values[0])

In [26]:
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

In [21]:
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#  Insert Record into Song Table

Implement the song_table_insert query in sql_queries.py

In [27]:
cur.execute(song_table_insert, song_data)
conn.commit()

# 2: Artists Table :
## Extract Data for Artists Table:
 - Select columns for artist ID, Name, location, latitude, and lngitude.
 - Use df.values to select just the values from the dataframes
 - Convert the array to a list and set it to artist_data

In [28]:
artist_data = list(df[['artist_id', 'artist_name','artist_location','artist_latitude','artist_longitude']].values[0])

In [29]:
artist_data


['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

In [31]:
cur.execute(artist_table_insert,artist_data)
conn.commit()

# Process Log_data
In this part, you will perform ETL on the second dataset, Log_data. To create the time and users dimensional table.
Lets perform ETL on a single log file and loa a single record into each table.


In [32]:
log_files = get_files('./data/log_data')

In [33]:
filepath = log_files[0]

In [34]:
log_files

['C:\\Users\\DELL\\Documents\\GitHub\\Data_Modeling_Postgres\\data\\log_data\\2018\\11\\2018-11-01-events.json',
 'C:\\Users\\DELL\\Documents\\GitHub\\Data_Modeling_Postgres\\data\\log_data\\2018\\11\\2018-11-02-events.json',
 'C:\\Users\\DELL\\Documents\\GitHub\\Data_Modeling_Postgres\\data\\log_data\\2018\\11\\2018-11-03-events.json',
 'C:\\Users\\DELL\\Documents\\GitHub\\Data_Modeling_Postgres\\data\\log_data\\2018\\11\\2018-11-04-events.json',
 'C:\\Users\\DELL\\Documents\\GitHub\\Data_Modeling_Postgres\\data\\log_data\\2018\\11\\2018-11-05-events.json',
 'C:\\Users\\DELL\\Documents\\GitHub\\Data_Modeling_Postgres\\data\\log_data\\2018\\11\\2018-11-06-events.json',
 'C:\\Users\\DELL\\Documents\\GitHub\\Data_Modeling_Postgres\\data\\log_data\\2018\\11\\2018-11-07-events.json',
 'C:\\Users\\DELL\\Documents\\GitHub\\Data_Modeling_Postgres\\data\\log_data\\2018\\11\\2018-11-08-events.json',
 'C:\\Users\\DELL\\Documents\\GitHub\\Data_Modeling_Postgres\\data\\log_data\\2018\\11\\2018-11-

In [36]:
df = pd.read_json(filepath,lines = True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


#  3: time Table
## Extract Data for Time Table:
 - Filter records by NextSong action.
 - Convert the ts timestamp column to datetime
     - Hint : The current timestamp is in milliseconds
 - Extract the timestamp, hour, day, week of year, and weekday from the ts column and set time_data to a list containing these values

In [37]:
df = df[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [46]:
t = pd.to_datetime(df['ts'], unit = 'ms')
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
t.head()

2   2018-11-01 21:01:46.796
4   2018-11-01 21:05:52.796
5   2018-11-01 21:08:16.796
6   2018-11-01 21:11:13.796
7   2018-11-01 21:17:33.796
Name: ts, dtype: datetime64[ns]

In [47]:
from datetime import datetime

time_data = (t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ('ts', 'hour', 'day', 'week', 'month', 'year', 'weekday')

<ipython-input-47-2c6376c5d9d2>:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  time_data = (t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday)


In [48]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

,ts,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796,21,1,44,11,2018,3
5,2018-11-01 21:08:16.796,21,1,44,11,2018,3
6,2018-11-01 21:11:13.796,21,1,44,11,2018,3
7,2018-11-01 21:17:33.796,21,1,44,11,2018,3


#  Insert Recod into Time Table
Implement the time_table_insert query in sql_queries.py and run the cell below to insert records for the timestamps in this log file into the time table.

In [49]:
for i,row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

# 4: USERS Table
## Etract Data for users Table
 - Select columns for user ID, first name, last name, gender and level and set to user_df
 

In [52]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df.head()

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free


#  Insert Record into Users Table

Implement the user_table_insert query in sql_queries.py and run the cell bellow to insert records for users in this log file into the users table

In [53]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

#  Songplays Table : 
Extract Data and songplays Table 
This one is file are all needed for the SongPlays table. Since The log file does not specify an ID for either the song or the artist.
 - Implement the song_select query in sql_queries.py to find song Id and artist Id based on the title, Artist name, and duration of a song.
 - Select the timeStamp, userID, level, songId, Artist ID, Session ID, location, And user agent and set to songplay_data
 

In [ ]:
for index, row in df.iterrows(0):
    # gete songid and artistid from song and artist tables
    cur